In [1]:
import torchio as tio
import os
import os.path as path

In [2]:
fp=f'../../data/original'
fp3=f'{fp}/3T'
fp7=f'{fp}/7T'
fpm=f'{fp}/mask'

with open('../../data/subjects.txt', 'r') as fp:
  fns = [line.strip() for line in fp]
validation_subjects=['05','9015']


postfix='.nii.gz'

In [3]:
#load all subjects with torchio
subjects=[]
for i in range(len(fns)):
    #normalize value between 0,1
    #outlier removeal done during preprocessing (5%-95% percentile kept)
    rescale1=tio.RescaleIntensity(out_min_max=(0, 1),percentiles=(0,100))
    rescale2=tio.RescaleIntensity(out_min_max=(0, 1),percentiles=(0,100))
    subject=tio.Subject(
        t1_3T = rescale1(tio.ScalarImage(path.join(fp3,str(fns[i])+postfix))),
        t1_7T = rescale2(tio.ScalarImage(path.join(fp7,str(fns[i])+postfix))),
        mask = tio.LabelMap(path.join(fpm,str(fns[i])+postfix))
    )
    subjects.append(subject)
# org_dataset = tio.SubjectsDataset(subjects)
print('Dataset size:', len(subjects), 'subjects')

Dataset size: 18 subjects


# data augmentation

In [5]:
transform = tio.Compose([
    # Coordinate changes for 3T & 7T
    tio.RandomFlip(axes=('LR','AP'),
                    p=1,
                    include=('t1_3T','t1_7T', 'mask')),
    tio.RandomAffine(scales=0, #no scaling, data in MNI space
                    degrees=20,
                    isotropic=True,
                    p=1,
                    include=('t1_3T','t1_7T', 'mask')),
    # Elastic deformation may cause some resolution degradation
    tio.RandomElasticDeformation(num_control_points=7,
                    p=1,
                    include=('t1_3T','t1_7T', 'mask')),  
    # image alterations for 3T only
    tio.RandomGamma(log_gamma=(-0.3,0.3),
                    p=1,
                    include=('t1_3T')),
    # allow image downsampling in each dimension (3T only)
    ### Random anisotropy causes shift!
    # do resample and then resample back!
    tio.RandomAnisotropy(axes=(0),
                         downsampling=(1, 5),
                         p=1,
                         include=('t1_3T')),
    tio.RandomAnisotropy(axes=(1),
                         downsampling=(1, 5),
                         p=1,
                         include=('t1_3T')),
    tio.RandomAnisotropy(axes=(2),
                         downsampling=(1, 5),
                         p=1,
                         include=('t1_3T')),                                        
])

In [6]:
subject_set = tio.SubjectsDataset(subjects)
for i, subject in enumerate(subject_set):
  transformed_subject = transform(subject)
  
  tind = '3'
  directory3t = f'../../data/transform{tind}/3T'
  directory7t = f'../../data/transform{tind}/7T'
  directorymask = f'../../data/transform{tind}/mask'
  if not os.path.isdir(directory3t):
    os.makedirs(directory3t)
    os.makedirs(directory7t)
    os.makedirs(directorymask)

  
  transformed_subject.t1_3T.save(f'{directory3t}/{fns[i]}_transformed.nii.gz')
  transformed_subject.t1_7T.save(f'{directory7t}/{fns[i]}_transformed.nii.gz')
  transformed_subject.mask.save(f'{directorymask}/{fns[i]}_transformed.nii.gz')

# downsample

In [7]:
scale_factor = 2
resampled_subjects = []
for subject in subjects:
    spacing = subject.t1_3T.spacing
    new_spacing = [s * scale_factor for s in spacing]
    resample_down = tio.Resample(new_spacing)
    resample_up = tio.Resample(spacing)
    resampled_subject = resample_down(subject)
    resampled_subject = resample_up(resampled_subject)
    resampled_subjects.append(resampled_subject)

In [11]:
resampled_set = tio.SubjectsDataset(resampled_subjects)
directory3t = f'../../data/down{scale_factor}/3T'
directorymask = f'../../data/down{scale_factor}/mask'

if not os.path.isdir(directory3t):
  os.makedirs(directory3t)
  os.makedirs(directorymask)

for i, subject in enumerate(resampled_set):
    subject.t1_3T.save(f'{directory3t}/{fns[i]}_down{scale_factor}.nii.gz')
    subject.mask.save(f'{directorymask}/{fns[i]}_down{scale_factor}.nii.gz')